In [16]:
# Prime modulus for the elliptic curve (large prime number defining field F_p)
P = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
# Elliptic curve parameter a (we are using curve equation y^2 = x^3 + ax + b over F_p, here a=0)
A = 0
# Elliptic curve parameter b (as per curve equation, here b=7)
B = 7
# Generator point (G) on the curve, typically a well-known starting point for scalar multiplication
G = (2, 22)  # Example coordinates
# Order of the curve (number of points on the curve)
ORDER = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# Function to compute the modular inverse of a number x under a given modulus
def mod_inv(x, modulus):
    if x == 0:
        raise ValueError("Cannot compute modular inverse of zero.")
    return pow(x, modulus - 2, modulus)

# Point addition on elliptic curve using modulus ORDER
def point_addition(P, Q):
    if P == (0, 0):
        return Q
    if Q == (0, 0):
        return P
    if P == Q:
        if P[1] == 0:
            return (0, 0)
        slope = (3 * P[0] * P[0] + A) * mod_inv(2 * P[1], ORDER) % ORDER
    else:
        if P[0] == Q[0]:
            return (0, 0)
        slope = (Q[1] - P[1]) * mod_inv(Q[0] - P[0], ORDER) % ORDER
    x_r = (slope * slope - P[0] - Q[0]) % ORDER
    y_r = (slope * (P[0] - x_r) - P[1]) % ORDER
    return (x_r, y_r)

# Scalar multiplication using modulus ORDER
def scalar_multiplication(k, P):
    result = (0, 0)
    addend = P
    while k:
        if k & 1:
            result = point_addition(result, addend)
        addend = point_addition(addend, addend)
        k >>= 1
    return result

# Deriving keys based on identity (without hashing)
def derive_key_from_identity(identity):
    identity_numeric = sum(ord(c) for c in identity) % ORDER
    private_key = identity_numeric
    public_key = scalar_multiplication(private_key, G)
    return private_key, public_key

# Deriving shared secret directly from elliptic curve point
def shared_secret(private_key, public_key):
    shared = scalar_multiplication(private_key, public_key)
    return shared[0] % 256  # Using only x-coordinate modulo 256

# Simple XOR encryption/decryption function
def xor_encrypt_decrypt(data, key):
    if isinstance(data, str):
        data = data.encode('utf-8')  # Convert string data to bytes
    key_stream = [key % 256 for _ in range(len(data))]
    encrypted_bytes = bytes([data[i] ^ key_stream[i] for i in range(len(data))])
    return encrypted_bytes

# Usage demonstration
identity = "abc@xyz.com"
payment_data = "Amount: ₹1234, To: USER"

# Derive keys for the user and central authority (CA)
user_private_key, user_public_key = derive_key_from_identity(identity)
ca_private_key, ca_public_key = derive_key_from_identity("CA")

# Generate shared secret for encryption key
encryption_key = shared_secret(user_private_key, ca_public_key)

# Encrypt the payment data
encrypted_data_bytes = xor_encrypt_decrypt(payment_data, encryption_key)
print("Encrypted:", encrypted_data_bytes)

# Decrypt the encrypted data using the shared secret
decryption_key = shared_secret(ca_private_key, user_public_key)
decrypted_data_bytes = xor_encrypt_decrypt(encrypted_data_bytes, decryption_key)

# Convert decrypted bytes back to string (UTF-8)
decrypted_data = decrypted_data_bytes.decode('utf-8')
print("Decrypted:", decrypted_data)

Encrypted: b'~RPJQK\x05\x1f\xdd\xbd\x86\x0e\r\x0c\x0b\x13\x1fkP\x05\x1fjlzm'
Decrypted: Amount: ₹1234, To: USER
